# How to use OpenCV to segment an image with the region growing algorithm

## Pseudo code

```cpp
seed_point // coordinates of the starting point
visited_matrix // boolean array/matrix, same size as image
               // all the pixels are initialised to false
point_list // empty list

point_list.enqueue( seed_point ) // Add the seed to the list

while ( point_list is not empty )
{
    this_point = point_list.dequeue() // Get a point from the list
    visited_matrix( this_point ) = true // Visit the point

    for each neighbour of this_point
    {
        if not visited_matrix( neighbour ) and neighbour is similar to this_point and neighbour not in point_list
        {
            point_list.enqueue( neighbour )
        }
    }
}
```

## Package inclusion for Python

In [1]:
from copy import deepcopy
import numpy as np
import cv2

## Read the image from a file on the disk and return a new matrix

![CT slice](../CT.png)

In [2]:
ct_slice = cv2.imread("../CT.png", cv2.IMREAD_GRAYSCALE)

## Check for errors

In [3]:
# Check for failure
if ct_slice is None: 
    raise Exception("Could not open or find the image")

## Display the image and pickup seeds

In [4]:
seed_set = [] # Stores the seeds

In [5]:
def mouseCallback(event, x, y, flags, param):
    global seed_set
    if  event == cv2.EVENT_LBUTTONDOWN:
        seed_set.append([x, y])
        colour = ( 0, 0, 255 )
        cv2.circle( param, (x, y), 4, colour, cv2.FILLED )
        cv2.imshow("CT slice", param)

In [6]:
colour_ct_slice = cv2.cvtColor(ct_slice, cv2.COLOR_GRAY2RGB);

cv2.namedWindow("CT slice", cv2.WINDOW_AUTOSIZE) # Create a window
cv2.imshow("CT slice", colour_ct_slice) # Show our image inside the created window
cv2.setMouseCallback("CT slice", mouseCallback, colour_ct_slice) # Register the callback function
cv2.waitKey(0) # Wait for any keystroke in the window

113

Here we selected 3 seedsto segment the lungs

![CT slice and the 3 seeds](../opencv-cxx-region_growing1.png)

## Region growing function

In [7]:
def regionGrowing(anImage,
                  aSeedSet,
                  anInValue=255,
                  tolerance=5):

    # boolean array/matrix, same size as image
    # all the pixels are initialised to false
    visited_matrix = np.zeros(anImage.shape).astype(np.uint8)

    # List of points to visit
    point_list = deepcopy(aSeedSet)

    # The list is not empty
    while len(point_list):

        # Get a point from the list
        this_point = point_list[-1]
        point_list = point_list[:-1]
        
        x = this_point[0]
        y = this_point[1]
        pixel_value = anImage[y][x]
                                                                                                                            
        # Visit the point
        visited_matrix[y][x] = anInValue

        # for each neighbour of this_point
        for j in range(y-1, y+2):

            # vertical index is valid
            if 0 <= j and j < anImage.shape[0]:
            
                for i in range(x-1, x+2):
                    
                    # hozirontal index is valid
                    if 0 <= i and i < anImage.shape[1]:

                        neighbour_value = float(anImage[j, i])
                        neighbour_visited = visited_matrix[j, i]
                        
                        if not neighbour_visited and \
                            abs(neighbour_value - pixel_value) <= (tolerance / 100.0 * 255.0): # neighbour is similar to this_point

                            point_list.append([i, j])

    return visited_matrix

## Segment the input

In [8]:
segmented_image = regionGrowing(ct_slice, seed_set, 255, 2)

## Display the result

In [9]:
cv2.namedWindow("Segmentation", cv2.WINDOW_AUTOSIZE) # Create a window
cv2.imshow("Segmentation", segmented_image) # Show our image inside the created window

cv2.waitKey(0) # Wait for any keystroke in the window
cv2.destroyAllWindows() # Destroy the created windows

Here is the result

![Segmentation](../opencv-cxx-region_growing2.png)

## Apply the mask

In [11]:
segmented_with_mask = cv2.bitwise_and(segmented_image, ct_slice)

In [13]:
cv2.namedWindow("CT image", cv2.WINDOW_GUI_EXPANDED) # Create a window
cv2.imshow("CT image", ct_slice) # Show our image inside the created window.

cv2.namedWindow("Segmentation", cv2.WINDOW_GUI_EXPANDED) # Create a window
cv2.imshow("Segmentation", segmented_image) # Show our image inside the created window.

cv2.namedWindow("Just the cells", cv2.WINDOW_GUI_EXPANDED) # Create a window
cv2.imshow("Just the lungs", segmented_with_mask) # Show our image inside the created window.

cv2.waitKey(0) # Wait for any keystroke in the window
cv2.destroyAllWindows() # Destroy all the created windows

You'll see:

![opencv-cxx-threshold4.png](../opencv-cxx-threshold4.png)

The background has been removed.